In [36]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame
import os
import joblib
import sklearn
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Import plotting libraries
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt

# Set larger fontsize for all plots
matplotlib.rcParams.update({'font.size': 30})
from IPython.display import clear_output

In [37]:
# Getting dataset
df = pd.read_csv('../raw data.csv')
df

,DOI,method,metal,metal_source,metal_amount_mmol,ligand,ligand_amount_mmol,reductant,reductant_amount_mmol,solvent1,sol1_vol_mL,solvent2,solv2_vol_mL,time_min,temp_c,size_nm,Absorbance,Ellip_mdeg,g_factor*10^4,g_factor*10^4.1
0,10.1021/cm3033725,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,N-acetyl-L-cysteine,0.5300,sodium borohydride,1.3200,water,32.5,None,0.0,720,0,3.06,1.1,0,None,0
1,None,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,N-acetyl-L-cysteine,0.5300,sodium borohydride,1.3200,water,33.5,None,0.0,720,0,1,1,2.5,None,0.757989206
2,None,one-pot reduction,silver,silver nitrate,0.870,glutathione,0.9700,sodium borohydride,9.9900,water,30.0,None,0.0,60,0,1,1.3,2.5,None,0.58306862
3,None,one-pot reduction,silver,silver nitrate,0.930,N-acetyl-Lcysteine,1.0000,sodium borohydride,10.2000,water,37.0,None,0.0,60,0,2.37,2,0,None,0
4,10.1002/adfm.201001120,one-pot reduction,gold,Tetrachloroauric(III) acid,0.132,glutathione,0.5300,sodium borohydride,1.3200,water,32.5,None,0.0,720,0,4,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,None,one-pot reduction,silver,silver nitrate,0.200,Dihydrolipoic acid,0.0001,sodium borohydride,0.0005,water,9.0,None,0.0,480,25,None,None,None,2,2
197,10.1021/la703351p,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.5000,sodium borohydride,5.0000,water,30.0,methanol,25.0,90,25,1.05,None,None,15,15
198,None,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.5000,sodium borohydride,5.0000,water,30.0,methanol,25.0,90,25,1.3,None,None,10,10
199,None,one-pot reduction,silver,silver nitrate,0.500,penicillamine,0.2500,sodium borohydride,5.0000,water,30.0,methanol,25.0,90,25,1.05,None,None,7,7


In [38]:
#Checks if there are any columns with empty cells
df.isna().sum()

DOI                      0
method                   0
metal                    0
metal_source             0
metal_amount_mmol        0
ligand                   0
ligand_amount_mmol       0
reductant                0
reductant_amount_mmol    0
solvent1                 0
sol1_vol_mL              0
solvent2                 0
solv2_vol_mL             0
time_min                 0
temp_c                   0
size_nm                  0
Absorbance               0
Ellip_mdeg               0
g_factor*10^4            0
g_factor*10^4.1          0
dtype: int64

# Scaling and Transforming

In [39]:
df_input = df.drop(columns =['DOI','method','metal_source','size_nm','Absorbance','Ellip_mdeg','g_factor*10^4.1','g_factor*10^4'], inplace = False, axis = 1)
df_output = df[['size_nm','g_factor*10^4.1']]

In [40]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [41]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   metal                  201 non-null    object 
 1   metal_amount_mmol      201 non-null    float64
 2   ligand                 201 non-null    object 
 3   ligand_amount_mmol     201 non-null    float64
 4   reductant              201 non-null    object 
 5   reductant_amount_mmol  201 non-null    float64
 6   solvent1               201 non-null    object 
 7   sol1_vol_mL            201 non-null    float64
 8   solvent2               201 non-null    object 
 9   solv2_vol_mL           201 non-null    float64
 10  time_min               201 non-null    int64  
 11  temp_c                 201 non-null    int64  
dtypes: float64(5), int64(2), object(5)
memory usage: 19.0+ KB


In [42]:
#change temp and time from integer to float
df_input['temp_c'] = df_input['temp_c'].astype(float)
df_input['time_min'] = df_input['time_min'].astype(float)
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   metal                  201 non-null    object 
 1   metal_amount_mmol      201 non-null    float64
 2   ligand                 201 non-null    object 
 3   ligand_amount_mmol     201 non-null    float64
 4   reductant              201 non-null    object 
 5   reductant_amount_mmol  201 non-null    float64
 6   solvent1               201 non-null    object 
 7   sol1_vol_mL            201 non-null    float64
 8   solvent2               201 non-null    object 
 9   solv2_vol_mL           201 non-null    float64
 10  time_min               201 non-null    float64
 11  temp_c                 201 non-null    float64
dtypes: float64(7), object(5)
memory usage: 19.0+ KB


In [43]:
#Initializes 2 lists to contain all the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']
print(input_num_cols, input_cat_cols)

['metal_amount_mmol', 'ligand_amount_mmol', 'reductant_amount_mmol', 'sol1_vol_mL', 'solv2_vol_mL', 'time_min', 'temp_c'] ['metal', 'ligand', 'reductant', 'solvent1', 'solvent2']


In [44]:
#Initializes the ColumnTransformer object, and specifies what it will do with a dataframe
#scaling numerical columns
#onehotencoder creates a binary column for each categorical entry
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OrdinalEncoder( handle_unknown='error'), input_cat_cols)
], remainder = 'passthrough')
#what are the transformers in ct
ct.transformers

[('step1',
  StandardScaler(),
  ['metal_amount_mmol',
   'ligand_amount_mmol',
   'reductant_amount_mmol',
   'sol1_vol_mL',
   'solv2_vol_mL',
   'time_min',
   'temp_c']),
 ('step2',
  OrdinalEncoder(),
  ['metal', 'ligand', 'reductant', 'solvent1', 'solvent2'])]

In [45]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0
1,-0.454943,-0.187770,-0.196938,-0.103038,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0
2,1.601355,0.032059,0.273730,-0.174625,-0.525157,-0.593509,-0.997007,6.0,71.0,15.0,9.0,1.0
3,1.768534,0.047047,0.285130,-0.031450,-0.525157,-0.593509,-0.997007,6.0,47.0,15.0,9.0,1.0
4,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,71.0,15.0,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
196,-0.265474,-0.452514,-0.268570,-0.604149,-0.525157,0.217328,-0.176375,6.0,35.0,15.0,9.0,1.0
197,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0
198,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0
199,0.570420,-0.327661,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0


In [46]:
#Number of numerical columns
len(input_num_cols)

7

In [47]:
#renaming columns
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

In [48]:
df_input_scaled_encoded.columns

Index([    'metal_amount_mmol',    'ligand_amount_mmol',
       'reductant_amount_mmol',           'sol1_vol_mL',
                'solv2_vol_mL',              'time_min',
                      'temp_c',                       7,
                             8,                       9,
                            10,                      11],
      dtype='object')

In [49]:
#appends output columns
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

,metal_amount_mmol,ligand_amount_mmol,reductant_amount_mmol,sol1_vol_mL,solv2_vol_mL,time_min,temp_c,7,8,9,10,11,size_nm,g_factor*10^4.1
0,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0,3.06,0
1,-0.454943,-0.187770,-0.196938,-0.103038,-0.525157,0.680663,-0.997007,1.0,46.0,15.0,9.0,1.0,1,0.757989206
2,1.601355,0.032059,0.273730,-0.174625,-0.525157,-0.593509,-0.997007,6.0,71.0,15.0,9.0,1.0,1,0.58306862
3,1.768534,0.047047,0.285130,-0.031450,-0.525157,-0.593509,-0.997007,6.0,47.0,15.0,9.0,1.0,2.37,0
4,-0.454943,-0.187770,-0.196938,-0.123491,-0.525157,0.680663,-0.997007,1.0,71.0,15.0,9.0,1.0,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-0.265474,-0.452514,-0.268570,-0.604149,-0.525157,0.217328,-0.176375,6.0,35.0,15.0,9.0,1.0,None,2
197,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.05,15
198,0.570420,-0.202758,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.3,10
199,0.570420,-0.327661,0.002838,-0.174625,-0.006407,-0.535592,-0.176375,6.0,79.0,15.0,9.0,4.0,1.05,7


In [50]:
df_scaled_encoded.to_csv('dataset_scaled.csv')

# Making separate outputs dataset

In [51]:
#Saves into a list the row indexes to drop for size dataset
total_row_num = len(df_scaled_encoded)
drop_list_size =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['size_nm'].values[row_i] == 'None':
        drop_list_size.append(row_i)
    
#number of entries
print(total_row_num-len(drop_list_size))

144


In [52]:
#Drops rows that don't have size output
df_size_scaled_encoded = df_scaled_encoded.drop(drop_list_size)
df_size_scaled_encoded.to_csv('dataset_scaled_size.csv')